## Note:
This notebook was written before updates in integrationAnalysisTools, so probably won't work!!

In [14]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

import os, sys
import time
import tqdm
import torch
import IPython
import torchvision

import torch.nn as nn
import torch.nn.functional as F

from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder

from PIL import Image
from io import BytesIO

import alignmentAnalysisTools as iat

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
CPU = "cpu"
print(DEVICE)

cuda


In [3]:
dataPath = os.path.join('C:/', 'Users','andrew','Documents','machineLearning','datasets')

In [4]:
useNet = 'alexnet'
useWeights = None # 'DEFAULT' or None
if useNet=='resnet':
    resnet = torchvision.models.resnet18(weights=useWeights)
    resnet.to(DEVICE)
    preprocess = transforms.Compose([
                                 transforms.Resize(256),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                      std=[0.229, 0.224, 0.225]),
                                 ])
elif useNet=='alexnet':
    alexnet = torchvision.models.alexnet(weights=useWeights)
    alexnet.to(DEVICE)
    preprocess = transforms.Compose([
                                 transforms.Resize(256),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                      std=[0.229, 0.224, 0.225]),
                                 ])
elif useNet=='squeezenet':
    squeezenet = torchvision.models.squeezenet1_0(weights=useWeights)
    squeezenet.to(DEVICE)
    preprocess = transforms.Compose([
                                 transforms.Resize(256),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                      std=[0.229, 0.224, 0.225]),
                                 ])
elif useNet=='efficientnet':
    efficientnet = torchvision.models.efficientnet_b0(weights=useWeights)
    efficientnet.to(DEVICE)
    preprocess = transforms.Compose([
                                 transforms.Resize(256),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                      std=[0.229, 0.224, 0.225]),
                                 ])
else:
    raise ValueError("useNet string not recognized")    

In [5]:
dataSet = 'cifar10'
if dataSet=='cifar10':
    trainset = torchvision.datasets.CIFAR10(root=dataPath, train=True, download=True, transform=preprocess)
    testset = torchvision.datasets.CIFAR10(root=dataPath, train=False, download=True, transform=preprocess)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
    testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True, num_workers=2)
    numClasses = 10
    
elif dataSet=='cifar100':
    trainset = torchvision.datasets.CIFAR100(root=dataPath, train=True, download=True, transform=preprocess)
    testset = torchvision.datasets.CIFAR100(root=dataPath, train=False, download=True, transform=preprocess)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
    testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True, num_workers=2)
    numClasses = 100
    
elif dataSet=='imagenet':
    trainset = torchvision.datasets.ImageNet(root=imagenetPath, train=True, download=True, transform=preprocess)
    testset = torchvision.datasets.ImageNet(root=imagenetPath, train=False, download=True, transform=preprocess)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True, num_workers=2)
    testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=True, num_workers=2)
    numClasses = 1000
    
else:
    raise ValueError("Dataset not recognized")

Files already downloaded and verified
Files already downloaded and verified


In [6]:
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [7]:
# Define some hooks and return activations
t = time.time()

# Define Hook (will store activations of various levels in this dictionary)
activation = {}
def getActivation(name):
    # the hook signature
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

# Now, for whatever network I requested, return activations for each batch (since we have dropout, do it with repetitions)
numImages = 2**8

# Do it
if useNet=='resnet':
    None
    
elif useNet=='alexnet':
    im = []
    feat2 = []
    feat5 = []
    feat7 = []
    feat9 = []
    input1 = []
    linear1 = []
    input4 = []
    linear4 = []
    alexnet.classifier[0].p=0 #Add this to maximize estimate of correlations 
    alexnet.classifier[3].p=0 #Add this to maximize estimate of correlations 
    alexnet.features[1].inplace=False #Need this set to False to allow full activations (before ReLU)
    alexnet.classifier[2].inplace=False #Need this set to False to allow full activations (before ReLU)
    alexnet.classifier[5].inplace=False #Need this set to False to allow full activations (before ReLU)
    f2 = alexnet.features[2].register_forward_hook(getActivation('Feat2'))
    f5 = alexnet.features[5].register_forward_hook(getActivation('Feat5'))
    f7 = alexnet.features[7].register_forward_hook(getActivation('Feat7'))
    f9 = alexnet.features[9].register_forward_hook(getActivation('Feat9'))
    h1i = alexnet.classifier[0].register_forward_hook(getActivation('Dropout0')) # input to linear1
    h1 = alexnet.classifier[1].register_forward_hook(getActivation('Linear1')) # output of linear1
    h2i = alexnet.classifier[3].register_forward_hook(getActivation('Dropout3')) # input to linear4
    h2 = alexnet.classifier[4].register_forward_hook(getActivation('Linear4')) # output of linear4
    
    imCount = 0
    for batch in trainloader:
        images, labels = batch
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        imCount += images.shape[0]

        output = alexnet(images)
        im.append(images.to(CPU))
        feat2.append(activation['Feat2'].to(CPU))
        feat5.append(activation['Feat5'].to(CPU))
        feat7.append(activation['Feat7'].to(CPU))
        feat9.append(activation['Feat9'].to(CPU))
        input1.append(activation['Dropout0'].to(CPU))
        linear1.append(activation['Linear1'].to(CPU))
        input4.append(activation['Dropout3'].to(CPU))
        linear4.append(activation['Linear4'].to(CPU))
        if imCount >= numImages: 
            break
            
    # detach the hooks
    f2.remove()
    f5.remove()
    f7.remove()
    f9.remove()
    h1i.remove()
    h1.remove()
    h2i.remove()
    h2.remove()
    
    # Save summary variables
    allImages = torch.cat(im)
    allFeat2 = torch.cat(feat2)
    allFeat5 = torch.cat(feat5)
    allFeat7 = torch.cat(feat7)
    allFeat9 = torch.cat(feat9)
    allInput1 = torch.cat(input1)
    allOutput1 = torch.cat(linear1)
    allInput4 = torch.cat(input4)
    allOutput4 = torch.cat(linear4)
    weights1 = alexnet.classifier[1].weight.to(CPU).detach()
    weights4 = alexnet.classifier[4].weight.to(CPU).detach()
    print(f'Finished in {time.time() - t} seconds!')
    
    del im, feat2, feat5, feat7, feat9, input1, linear1, input4, linear4, activation
    del images, labels
    
elif useNet=='squeezenet':
    None

elif useNet=='efficientnet':
    None
    
else:
    raise ValueError("Didn't recognize useNet name (should be one of the above listed in if/elifs)")

Finished in 5.826122999191284 seconds!


In [8]:
print(allImages.shape)
print(allFeat2.shape)
print(allFeat5.shape)
print(allFeat7.shape)
print(allFeat9.shape)
print(allInput1.shape)
print(allOutput1.shape)
print(allInput4.shape)
print(allOutput4.shape)

torch.Size([1024, 3, 224, 224])
torch.Size([1024, 64, 27, 27])
torch.Size([1024, 192, 13, 13])
torch.Size([1024, 384, 13, 13])
torch.Size([1024, 256, 13, 13])
torch.Size([1024, 9216])
torch.Size([1024, 4096])
torch.Size([1024, 4096])
torch.Size([1024, 4096])


In [18]:
inputActivity = allImages
layer = alexnet.features[0]

t = time.time()
preprocess = transforms.Pad(layer.padding)
pInput = np.array(preprocess(inputActivity))
print(f"Time to pad: {time.time() - t} seconds.")

t = time.time()
hMax, wMax = iat.getMaximumStrides(inputActivity.shape[2], inputActivity.shape[3], layer)
numLooks = hMax * wMax
numChannels = layer.out_channels
intLayer = np.empty((numChannels,numLooks))
print(f"Time to preallocate integration matrix: {time.time() - t} seconds.")

t = time.time()
stride = (0,0)
numImages = pInput.shape[0]
wIdxStart, hIdxStart = np.meshgrid(np.arange(0,layer.kernel_size[0]), np.arange(0, layer.kernel_size[1]))
numElements = pInput.shape[1] * layer.kernel_size[0] * layer.kernel_size[1]
alignedInput = pInput[:,:,hIdxStart + stride[0]*layer.stride[0], wIdxStart + stride[1]*layer.stride[1]].reshape(numImages, numElements)
alignedWeights = layer.weight.reshape(layer.out_channels, numElements).to('cpu').detach()
print(alignedInput.shape)
print(alignedWeights.shape)
print(f"Time to prepare data for integration analysis: {time.time() - t} seconds.")

t = time.time()
alignedInput = torch.tensor(alignedInput)
idxMute = torch.where(torch.std(alignedInput,axis=0)==0)[0]
b,n = alignedInput.shape
m = alignedWeights.shape[1]
cc = torch.corrcoef(alignedInput.T)
cc[idxMute,:] = 0
cc[:,idxMute] = 0
rq = torch.sum(torch.matmul(alignedWeights,cc) * alignedWeights,axis=1) / torch.sum(alignedWeights*alignedWeights, axis=1)
rq = -torch.log(rq/n)
tOnce = time.time() - t
tEstimate = tOnce * numLooks
print(f"Time to measure integration once: {tOnce} seconds (size rq: {rq.shape}).")
print(f"Estimate over all looks: {tEstimate}.")

t = time.time()
intLayer = iat.similarityConvLayer(allImages, alexnet.features[0])
print(f"Time to measure integration across across all looks: {time.time() - t} seconds (size intLayer: {intLayer.shape}).")

Time to pad: 0.7594590187072754 seconds.
Time to preallocate integration matrix: 0.0 seconds.
(4096, 363)
torch.Size([64, 363])
Time to prepare data for integration analysis: 0.014950037002563477 seconds.
Time to measure integration once: 0.014949798583984375 seconds (size rq: torch.Size([64])).
Estimate over all looks: 45.223140716552734.
Time to measure integration across across all looks: 69.07755923271179 seconds (size intLayer: (64, 3025)).


In [10]:
# Compute integration for the convolutional layers
t = time.time()
simFeat0 = iat.similarityConvLayer(allImages, alexnet.features[0])
intFeat0 = iat.integration(simFeat0)
print(f"intFeat0 finished: {time.time()-t} seconds.")

t = time.time()
simFeat3 = iat.similarityConvLayer(allFeat2, alexnet.features[3])
intFeat3 = iat.integration(simFeat3)
print(f"intFeat3 finished: {time.time()-t} seconds.")

t = time.time()
simFeat6 = iat.similarityConvLayer(allFeat5, alexnet.features[6])
intFeat6 = iat.integration(simFeat6)
print(f"intFeat6 finished: {time.time()-t} seconds.")

t = time.time()
simFeat8 = iat.similarityConvLayer(allFeat7, alexnet.features[8])
intFeat8 = iat.integration(simFeat8)
print(f"intFeat8 finished: {time.time()-t} seconds.")

t = time.time()
simFeat10 = iat.similarityConvLayer(allFeat9, alexnet.features[10])
intFeat10 = iat.integration(simFeat10)
print(f"intFeat10 finished: {time.time()-t} seconds.")

intFeat0 finished: 17.113802433013916 seconds.
intFeat3 finished: 22.20647621154785 seconds.
intFeat6 finished: 5.3983283042907715 seconds.
intFeat8 finished: 14.49635624885559 seconds.
intFeat10 finished: 7.824768304824829 seconds.


In [11]:
# Compute integration for the feedforward layers
t = time.time()
simClass1 = iat.similarity(allInput1, weights1)
intClass1 = iat.integration(simClass1)
print(f"intClass1 finished: {time.time()-t} seconds.")

t = time.time()
simClass4 = iat.similarity(allInput4, weights4)
intClass4 = iat.integration(simClass4)
print(f"intClass4 finished: {time.time()-t} seconds.")

C:\Users\andrew\AppData\Local\Temp\ipykernel_10736\2090471768.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputActivity = torch.tensor(inputActivity)
C:\Users\andrew\AppData\Local\Temp\ipykernel_10736\2090471768.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  weights = torch.tensor(weights)


intClass1 finished: 1.4122743606567383 seconds.
intClass4 finished: 0.2890329360961914 seconds.


In [12]:
print(f'RQ-Feat0: mean={np.mean(intFeat0)}, std={np.std(intFeat0)}.')
print(f'RQ-Feat3: mean={np.mean(intFeat3)}, std={np.std(intFeat3)}.')
print(f'RQ-Feat6: mean={np.mean(intFeat6)}, std={np.std(intFeat6)}.')
print(f'RQ-Feat8: mean={np.mean(intFeat8)}, std={np.std(intFeat8)}.')
print(f'RQ-Feat10: mean={np.mean(intFeat10)}, std={np.std(intFeat10)}.')
print(f'RQ-Class1: mean={np.mean(intClass1)}, std={np.std(intClass1)}.')
print(f'RQ-Class4: mean={np.mean(intClass4)}, std={np.std(intClass4)}.')

RQ-Feat0: mean=6.371365611172658, std=1.1771545750873715.
RQ-Feat3: mean=7.662070023816609, std=0.6305830852814601.
RQ-Feat6: mean=7.674985692272407, std=0.4429299781592306.
RQ-Feat8: mean=8.340740360815586, std=0.37125329957116604.
RQ-Feat10: mean=7.95823271467885, std=0.3466117349915805.
RQ-Class1: mean=9.214461326599121, std=0.25561967492103577.
RQ-Class4: mean=8.512177467346191, std=0.22892655432224274.
